# Preprocessing

Здесь мы проведем предобработку данных, полученных на предыдущем шаге, в FileNormalizer.ipynb.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.options.display.max_columns = None

Файл с данными я буду брать прямо с диска. Иначе ссылку на файл пришлось бы обновлять после каждого запуска предыдущего шага.

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/fns_fiscal_1nf.csv')
data.head()

,protocolVersion,operationType,cashTotalSum,shiftNumber,counterSubmissionSum,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,postpaymentSum,fiscalDocumentNumber,prepaymentSum,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,provisionSum,sellerAddress,fiscalDocumentFormatVer,propertiesData,prepaidSum,paymentAgentType,fnsSite,creditSum,propertiesUser.propertyValue,propertiesUser.propertyName,authorityUri,fnsUrl,nds10,nds0,addressToCheckFiscalSign,nds20,retailAddress,ndsCalculated10,code,operatorPhoneToTransfer,operatorToReceivePhone,operatorTransferAddress,operatorTransferInn,operatorTransferName,paymentAgentOperation,paymentAgentPhone,providerPhone,message,stornoItems,modifiers,fnsSiteUrl,raw_data,propertiesUser_key,propertiesUser_value
0,2.0,1,0,24,0.0,50175,50175,8363.0,2020-09-16,3.0,1,556,1.0,onlineoplata@esplus.ru,0.0,23206,0.0,onlineoplata@esplus.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1,0,24,0.0,25420,25420,4237.0,2020-09-16,3.0,1,565,1.0,onlineoplata@esplus.ru,0.0,23215,0.0,onlineoplata@esplus.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,0,14,NaN,6000,6000,NaN,2020-09-16,3.0,0,529,NaN,NaN,NaN,75001,NaN,+79091082396,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1,0,14,NaN,10000,10000,NaN,2020-09-16,3.0,0,537,NaN,NaN,NaN,75009,NaN,+79092626865,1667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1,0,14,NaN,20000,20000,NaN,2020-09-16,3.0,0,553,NaN,NaN,NaN,75025,NaN,+79089895687,3333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#data.drop('Unnamed: 0', axis=1, inplace=True)

Для удобства некоторых подсчетов временно создадим копию данных, где все пропуски заполнены нулями.

Это, в том числе, позволяет вычислить корреляцию между столбцами.

In [ ]:
data_0 = data.fillna(0)

In [ ]:
corr = data_0.corr()
corr.style.background_gradient(cmap='coolwarm')

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,protocolVersion,operationType,cashTotalSum,shiftNumber,counterSubmissionSum,totalSum,ecashTotalSum,nds18,receiptCode,taxationType,requestNumber,internetSign,postpaymentSum,fiscalDocumentNumber,prepaymentSum,ndsCalculated20,ndsNo,ndsCalculated18,provisionSum,fiscalDocumentFormatVer,prepaidSum,paymentAgentType,creditSum,nds10,nds0,addressToCheckFiscalSign,nds20,ndsCalculated10,code,operatorTransferAddress,operatorTransferInn,operatorTransferName,paymentAgentOperation
protocolVersion,1.000000,-0.051573,-0.032292,0.041976,nan,-0.031655,0.001538,0.070845,0.072232,-0.004247,-0.172985,0.350253,nan,-0.041999,nan,-0.095786,-0.036353,-0.045825,nan,-0.210955,nan,-0.077420,nan,0.141169,-0.003973,nan,-0.033038,-0.021128,-0.072232,nan,nan,nan,nan
operationType,-0.051573,1.000000,-0.007730,0.019440,nan,0.061521,0.316190,-0.011019,0.015502,-0.038930,0.103085,-0.013734,nan,0.082236,nan,-0.020557,0.063694,-0.018246,nan,-0.045273,nan,-0.016615,nan,-0.015260,-0.007421,nan,-0.007090,-0.004534,-0.015502,nan,nan,nan,nan
cashTotalSum,-0.032292,-0.007730,1.000000,-0.022167,nan,0.975702,-0.015808,-0.017739,0.006927,0.029232,-0.046861,-0.032771,nan,-0.079387,nan,-0.014357,0.978984,-0.011933,nan,-0.027961,nan,-0.011661,nan,-0.003623,-0.004873,nan,-0.004952,-0.001194,-0.006927,nan,nan,nan,nan
shiftNumber,0.041976,0.019440,-0.022167,1.000000,nan,-0.029040,-0.031107,0.016422,0.105200,0.176796,-0.328853,-0.134464,nan,0.389542,nan,-0.152397,-0.030954,-0.037776,nan,-0.043378,nan,0.233785,nan,0.154788,0.032415,nan,0.034974,-0.019478,-0.105200,nan,nan,nan,nan
counterSubmissionSum,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
totalSum,-0.031655,0.061521,0.975702,-0.029040,nan,1.000000,0.203121,0.043938,0.013829,0.013674,-0.040999,-0.010452,nan,-0.078948,nan,-0.006897,0.995662,-0.000840,nan,-0.032319,nan,-0.012671,nan,0.003783,0.043412,nan,-0.005707,-0.003231,-0.013829,nan,nan,nan,nan
ecashTotalSum,0.001538,0.316190,-0.015808,-0.031107,nan,0.203121,1.000000,0.280472,0.031700,-0.067807,0.024444,0.100457,nan,-0.001670,nan,0.033416,0.168540,0.050173,nan,-0.021088,nan,-0.005178,nan,0.034013,0.220501,nan,-0.003686,-0.009272,-0.031700,nan,nan,nan,nan
nds18,0.070845,-0.011019,-0.017739,0.016422,nan,0.043938,0.280472,1.000000,0.054746,-0.134240,0.096053,0.093298,nan,0.022380,nan,-0.072598,-0.029496,-0.064439,nan,-0.047537,nan,-0.055663,nan,0.013244,-0.026208,nan,-0.025040,-0.016013,-0.054746,nan,nan,nan,nan
receiptCode,0.072232,0.015502,0.006927,0.105200,nan,0.013829,0.031700,0.054746,1.000000,-0.224574,0.099180,0.065797,nan,0.178690,nan,0.028791,0.007877,0.025555,nan,-0.342405,nan,0.038011,nan,0.021372,0.010394,nan,0.009930,0.006351,-1.000000,nan,nan,nan,nan
taxationType,-0.004247,-0.038930,0.029232,0.176796,nan,0.013674,-0.067807,-0.134240,-0.224574,1.000000,-0.215080,-0.165236,nan,-0.161355,nan,-0.125176,0.028063,-0.043989,nan,0.161217,nan,-0.062467,nan,-0.054957,-0.012281,nan,-0.043175,0.065687,0.224574,nan,nan,nan,nan


In [ ]:
from math import isnan

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 57 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   protocolVersion               155 non-null    float64
 1   operationType                 800 non-null    int64  
 2   cashTotalSum                  800 non-null    int64  
 3   shiftNumber                   800 non-null    int64  
 4   counterSubmissionSum          155 non-null    float64
 5   totalSum                      800 non-null    int64  
 6   ecashTotalSum                 800 non-null    int64  
 7   nds18                         280 non-null    float64
 8   dateTime                      800 non-null    object 
 9   receiptCode                   783 non-null    float64
 10  taxationType                  800 non-null    int64  
 11  requestNumber                 800 non-null    int64  
 12  internetSign                  142 non-null    float64
 13  sende

## ecashTotalSum + cashTotalSum == totalSum

Видно, что есть только 3 строки, где ecashTotalSum + cashTotalSum != totalSum. Во всех этих строках ecashTotalSum + cashTotalSum < totalSum и непонятно, откуда взялась оставшаяся сумма.

Похоже, в этих строках ошибка. Удалим их.

In [ ]:
temp_rows = data_0.query('not (ecashTotalSum + cashTotalSum == totalSum)')
temp_rows

,protocolVersion,operationType,cashTotalSum,shiftNumber,counterSubmissionSum,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,postpaymentSum,fiscalDocumentNumber,prepaymentSum,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,provisionSum,sellerAddress,fiscalDocumentFormatVer,propertiesData,prepaidSum,paymentAgentType,fnsSite,creditSum,propertiesUser.propertyValue,propertiesUser.propertyName,authorityUri,fnsUrl,nds10,nds0,addressToCheckFiscalSign,nds20,retailAddress,ndsCalculated10,code,operatorPhoneToTransfer,operatorToReceivePhone,operatorTransferAddress,operatorTransferInn,operatorTransferName,paymentAgentOperation,paymentAgentPhone,providerPhone,message,stornoItems,modifiers,fnsSiteUrl,raw_data,propertiesUser_key,propertiesUser_value
18,0.0,1,0,35,0.0,74700,0,12450.0,2020-09-16,3.0,1,284,0.0,ofd@reg.ru,0.0,51301,0.0,ofdredirect@reg.ru,0.0,0.0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0
730,0.0,1,45000,29,0.0,95000,0,0.0,2020-09-14,3.0,2,3,0.0,0,0.0,323,0.0,0,0.0,95000.0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0
743,0.0,1,0,29,0.0,254000,0,0.0,2020-09-14,3.0,2,4,0.0,0,0.0,324,0.0,0,0.0,254000.0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0


In [ ]:
temp_index = temp_rows.index

In [ ]:
data.drop(index=temp_index, inplace=True)

## Заполнение пропусков в столбцах с денежными суммами

Пропуски в столбцах, содержащих денежные суммы, логично заполнить нулями.

In [ ]:
not_na_count = data.apply(lambda x: x.count())

In [ ]:
nds_columns = list(filter(lambda x: 'nds' in x, data.columns))
nds_columns

['nds18',
 'ndsCalculated20',
 'ndsNo',
 'ndsCalculated18',
 'nds10',
 'nds0',
 'nds20',
 'ndsCalculated10']

In [ ]:
data[nds_columns] = data[nds_columns].fillna(0)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 57 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   protocolVersion               155 non-null    float64
 1   operationType                 797 non-null    int64  
 2   cashTotalSum                  797 non-null    int64  
 3   shiftNumber                   797 non-null    int64  
 4   counterSubmissionSum          155 non-null    float64
 5   totalSum                      797 non-null    int64  
 6   ecashTotalSum                 797 non-null    int64  
 7   nds18                         797 non-null    float64
 8   dateTime                      797 non-null    object 
 9   receiptCode                   780 non-null    float64
 10  taxationType                  797 non-null    int64  
 11  requestNumber                 797 non-null    int64  
 12  internetSign                  142 non-null    float64
 13  sende

## Удаление столбцов, содержащих только 0 и NaN

Некоторые столбцы содержат только 0 и NaN. Удалим эти столбцы.

In [ ]:
data['prepaymentSum'].value_counts()

0.0    155
Name: prepaymentSum, dtype: int64

In [ ]:
data['postpaymentSum'].value_counts()

0.0    155
Name: postpaymentSum, dtype: int64

In [ ]:
data['counterSubmissionSum'].value_counts()

0.0    155
Name: counterSubmissionSum, dtype: int64

In [ ]:
data['prepaidSum'].value_counts()

0.0    125
Name: prepaidSum, dtype: int64

In [ ]:
data['creditSum'].value_counts()

0.0    125
Name: creditSum, dtype: int64

In [ ]:
data['provisionSum'].value_counts()

0.0    125
Name: provisionSum, dtype: int64

In [ ]:
data.drop(['prepaymentSum', 'postpaymentSum', 'counterSubmissionSum', 'prepaidSum', 'creditSum', 'provisionSum'], axis=1, inplace=True)

## Удаление столбцов, содержащих только NaN

Есть столбцы, в которых все значения равны NaN. Удалим их.

In [ ]:
data.drop(['addressToCheckFiscalSign', 'operatorTransferAddress', 'operatorTransferInn', 'operatorTransferName', 'paymentAgentOperation'], axis=1, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   protocolVersion               155 non-null    float64
 1   operationType                 797 non-null    int64  
 2   cashTotalSum                  797 non-null    int64  
 3   shiftNumber                   797 non-null    int64  
 4   totalSum                      797 non-null    int64  
 5   ecashTotalSum                 797 non-null    int64  
 6   nds18                         797 non-null    float64
 7   dateTime                      797 non-null    object 
 8   receiptCode                   780 non-null    float64
 9   taxationType                  797 non-null    int64  
 10  requestNumber                 797 non-null    int64  
 11  internetSign                  142 non-null    float64
 12  senderAddress                 134 non-null    object 
 13  fisca

## Удаление столбцов, содержащих только пустые массивы и NaN

Ряд столбцов содержат только пустые массивы и NaN. Удалим их.

In [ ]:
data['providerPhone'].value_counts()

[]    17
Name: providerPhone, dtype: int64

In [ ]:
data['paymentAgentPhone'].value_counts()

[]    17
Name: paymentAgentPhone, dtype: int64

In [ ]:
data['operatorToReceivePhone'].value_counts()

[]    17
Name: operatorToReceivePhone, dtype: int64

In [ ]:
data['operatorPhoneToTransfer'].value_counts()

[]    17
Name: operatorPhoneToTransfer, dtype: int64

In [ ]:
data['message'].value_counts()

[]    3
Name: message, dtype: int64

In [ ]:
data['modifiers'].value_counts()

[]    3
Name: modifiers, dtype: int64

In [ ]:
data['stornoItems'].value_counts()

[]    3
Name: stornoItems, dtype: int64

In [ ]:
data.drop(['providerPhone', 'paymentAgentPhone', 'operatorPhoneToTransfer', 'message', 'modifiers', 'stornoItems', 'operatorToReceivePhone'], axis=1, inplace=True)

## Удаление столбцов, содержащих бинарные данные

Несколько столбцов содержат уникальные бинарные данные. Удалим эти столбцы.

In [ ]:
data['raw_data'].value_counts()

//8AHKUDARAJklFEAwAWUxUA4y1ErixUBAC4zQAGARUDABEBEQQQADkyNTE0NDAzMDAxNjUzMTUNBBQAMDAwMjgwMTgxMDAwOTgxMCAgICD6AwwANzc0MDAwMDA3NiAgEAQEAC3jAAD0AwQA3ApfXzUEBgAxBIptQCIOBAQAEgAAABIEBABQBAAAHgQBAAH8AwIAiBMjBDEABgQMAJPhq+OjqCDhou+nqDcEAgCIE/8DAgAAARMEAgCIE68EAQABvgQBAAO8BAEABB8EAQABBwQBAAA5BAIAiBO/BAEAAMAEAQAAwQQBAAAkBAwAd3d3Lm5hbG9nLnJ1TgQCAEEDPAQoAD0EDgCNrqyl4CCur6XgoOaoqD4EEgCFkY+POjE0NzE3NzM5NjMzMDEMBAEAM7kEAQACgQbiAAs/4H4=                                                    1
//8AHKUDARAJklFEAwAViUUAGv62Xk57zwAAAgAGARQDABABEQQQADkyNTE0NDAzMDAxNTg5NDUNBBQAMDAwMjc4MjEzODA1NDYwNSAgICD6AwwANzc0MDAwMDA3NiAgEAQEAP4aAAD0AwQAEBNfXzUEBgAxBEQfm10OBAQAAwAAABIEBABSBAAAHgQBAAH8AwIA6AMjBDEABgQMAJPhq+OjqCDhou+nqDcEAgDoA/8DAgAAARMEAgDoA68EAQABvgQBAAO8BAEABB8EAQABBwQBAAA5BAIA6AO/BAEAAMAEAQAAwQQBAAAkBAwAd3d3Lm5hbG9nLnJ1TgQBAKc8BCgAPQQOAI2urKXgIK6vpeCg5qioPgQSAIWRj486MTQ3MTc4MTcyMjEwMQwEAQAxuQQBAAKBBoRhPdEBAg==                                                    1
//8AHKUDARAJklFEAwAWUzMBFkxGji4xcAECvwAGAToD

In [ ]:
data['propertiesData'].value_counts()

ws:CCZKNHAUGP    1
ws:CYPJ3EZV4J    1
ws:CPH44TE6Q3    1
ws:C255BUGQVJ    1
ws:CTTBRYUWTQ    1
ws:CNDD6ZU673    1
ws:C3QLMUXCWI    1
ws:CHZ2RKU4ZK    1
ws:CH2QK3CRB7    1
ws:CRHS5R3DBA    1
ws:CYOFZD25UZ    1
ws:CBXZEBZXHP    1
ws:CYLXDRUXVZ    1
ws:C4ICGQFFBY    1
ws:CGRTKRM2QI    1
ws:CFQQHLT6KL    1
ws:CB4NU6EW6U    1
ws:C5NBISJU3Z    1
ws:C34DEZH3LM    1
ws:CKUV2TTIWX    1
Name: propertiesData, dtype: int64

In [ ]:
data['propertiesUser.propertyValue'].value_counts()

f2810727730248b630fdd401470ef841    1
e427fd559278b6f9e3541c03536691a5    1
7bd4d9869245dfb936850a181123f22d    1
d825d8172376b1a565f94b09ec181ef0    1
d731322a22cad4dbc661c90633fabc0d    1
ac3f20f7f282f0cc754d758ca1f842fc    1
d287d783a3f43c5db128fd042307d684    1
defc66aff2f186a9f4f6ba97e17d30ef    1
23dbe3f7e39dab071789a7193064238e    1
428f83e3834a74de0523bf8360115287    1
9b03292c0326a751a6d0c08e1a285982    1
d3f45b6a023ef3f824510a1ced65efe1    1
1b909c8492a093cd550236092f55a854    1
7c4cccf953479503b6ef361bb1a1da05    1
ae9a6ef36205607cf7cfc68ae2a5e57f    1
ee056ae1a2854bd3762e0c826895e830    1
ea018ec122e94f1f7756978f69855b69    1
413ed01b3272a9d20067b90df067ca49    1
be32f68b22b46dfc363f1a810c9fd325    1
f11228c3a2774936e405a21cdda6c0b7    1
Name: propertiesUser.propertyValue, dtype: int64

In [ ]:
data.drop(['raw_data', 'propertiesData', 'propertiesUser.propertyValue'], axis=1, inplace=True)

## Удаление столбцов, содержащих адрес сайта ФНС (Федеральной налоговой службы)

В столбцах 'fnsSiteUrl' и 'fnsSite' "разные" значения, представляющие собой один и тот же адрес сайта ФНС. Адрес сайта ФНС - общеизвестная информация и не имеет отношения к данным, содержащимся в строках.

Удалим эти столбцы.

In [ ]:
data['fnsUrl'].value_counts()

www.nalog.ru             26
https://www.nalog.ru/     1
Name: fnsUrl, dtype: int64

In [ ]:
data['fnsSiteUrl'].value_counts()

www.nalog.ru    49
nalog.ru         1
Name: fnsSiteUrl, dtype: int64

In [ ]:
data['fnsSite'].value_counts()

nalog.ru                 20
www.nalog.ru              2
https://www.nalog.ru/     1
Name: fnsSite, dtype: int64

In [ ]:
data['authorityUri'].value_counts()

www.nalog.ru             47
https://www.nalog.ru/     1
Name: authorityUri, dtype: int64

In [ ]:
data.drop(['fnsUrl', 'fnsSiteUrl', 'fnsSite', 'authorityUri'], axis=1, inplace=True)

Как мы определили ранее, cashTotalSum + ecashTotalSum = totalSum.

Один из этих столбцов нужно удалить. При этом totalSum содержит наиболее важную саму по себе информацию, которая, скорее всего, коррелирует с другими признаками. Так что я решил удалить столбец cashTotalSum.

In [ ]:
data.drop('cashTotalSum', axis=1, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   protocolVersion              155 non-null    float64
 1   operationType                797 non-null    int64  
 2   shiftNumber                  797 non-null    int64  
 3   totalSum                     797 non-null    int64  
 4   ecashTotalSum                797 non-null    int64  
 5   nds18                        797 non-null    float64
 6   dateTime                     797 non-null    object 
 7   receiptCode                  780 non-null    float64
 8   taxationType                 797 non-null    int64  
 9   requestNumber                797 non-null    int64  
 10  internetSign                 142 non-null    float64
 11  senderAddress                134 non-null    object 
 12  fiscalDocumentNumber         797 non-null    int64  
 13  buyerAddress        

## Заполнение пропусков в категориальных столбцах с помощью "Missing"

Пропуски в остальных столбцах заполним значениями 'Missing'

In [ ]:
data['protocolVersion'].value_counts()

2.0    155
Name: protocolVersion, dtype: int64

In [ ]:
data['receiptCode'].value_counts()

3.0    780
Name: receiptCode, dtype: int64

In [ ]:
data['internetSign'].value_counts()

1.0    133
0.0      9
Name: internetSign, dtype: int64

In [ ]:
data['senderAddress'].value_counts()

noreply@ofd.ru                 57
onlineoplata@esplus.ru         35
echeck@1-ofd.ru                10
no-replay@bashesk.ru            9
ofdsupport@nvg.ru               7
sekretar.sdk@mosgortrans.ru     4
ofd@magnit.ru                   3
noreplyofd@kontur.ru            3
ofd@reg.ru                      2
it_info@rcurala.ru              2
nalogi@ttk.ru                   1
kassa@adeal.ru                  1
Name: senderAddress, dtype: int64

In [ ]:
data['buyerAddress'].value_counts()

onlineoplata@esplus.ru       35
+74912410085                  9
receipt-online@bashesk.ru     9
none                          4
ofd@itpc.ru                   3
                             ..
+79674011504                  1
+79020671807                  1
+79064491332                  1
veronsd1983@gmail.com         1
+79505252508                  1
Name: buyerAddress, Length: 202, dtype: int64

In [ ]:
data['userProperty.key'].value_counts()

АВТОБУС (городской маршрут)    20
Лицевой счет                    1
#69@1                           1
#87@1/2                         1
#90@1/2                         1
#95@1/2                         1
Name: userProperty.key, dtype: int64

In [ ]:
data['userProperty.value'].value_counts()

№ транспортного средства 552    6
№ транспортного средства 045    3
№ транспортного средства 682    3
№ транспортного средства 465    2
№ транспортного средства 152    2
№ транспортного средства 324    1
№ транспортного средства 217    1
381095810                       1
№ транспортного средства 153    1
№ транспортного средства 892    1
#2501602@7040                   1
#2502114@2100/7040              1
#2502114@7187/2222              1
#2501502@7156/2411              1
Name: userProperty.value, dtype: int64

In [ ]:
data['sellerAddress'].value_counts()

kassa@vivasoft.pro          28
noreply@chek.pofd.ru        25
support@go.yandex.com       17
post@ettu.ru                 6
support@support-uber.com     3
no-reply@ofd.yandex.ru       1
Name: sellerAddress, dtype: int64

In [ ]:
data['fiscalDocumentFormatVer'].value_counts()

2.0    125
Name: fiscalDocumentFormatVer, dtype: int64

In [ ]:
data['paymentAgentType'].value_counts()

 64.0    20
-1.0     17
Name: paymentAgentType, dtype: int64

In [ ]:
data['propertiesUser.propertyName'].value_counts()

trust_purchase_token    20
Name: propertiesUser.propertyName, dtype: int64

In [ ]:
data['retailAddress'].value_counts()

127410, Москва г, Алтуфьевское ш, дом № 33Г                                                                                         23
Ростовская область, г. Батайск, Северный массив, д. 5/4                                                                              2
 677007, Республика Саха (Якутия), г. Якутск, ул.Красильникова,3Д                                                                    1
Ростовская область, г. Новочеркасск, пр. Ермака, д. 88                                                                               1
Ростовская область, г. Ростов-на-Дону, Ленинский р-н, пр. Буденновский, д. 42/50                                                     1
143200, МО, г.Можайск, ул.Московская, д.22                                                                                           1
Name: retailAddress, dtype: int64

In [ ]:
data['code'].value_counts()

3.0    17
Name: code, dtype: int64

In [ ]:
data['propertiesUser_key'].value_counts()

Маршрут    20
Name: propertiesUser_key, dtype: int64

In [ ]:
data['propertiesUser_value'].value_counts()

ВЛАДИМИР (ЮГО-ЗАПАДНАЯ) Ц    19
ВЛАДИМИР - СОДЫШКА            1
Name: propertiesUser_value, dtype: int64

In [ ]:
fill_missing = ['protocolVersion', 'receiptCode', 'internetSign', 'senderAddress', 'buyerAddress', 'userProperty.key', 'userProperty.value', 'sellerAddress', 'fiscalDocumentFormatVer',\
                'paymentAgentType', 'propertiesUser.propertyName', 'retailAddress', 'code', 'propertiesUser_key', 'propertiesUser_value']

In [ ]:
data[fill_missing] = data[fill_missing].fillna('Missing')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   protocolVersion              797 non-null    object 
 1   operationType                797 non-null    int64  
 2   shiftNumber                  797 non-null    int64  
 3   totalSum                     797 non-null    int64  
 4   ecashTotalSum                797 non-null    int64  
 5   nds18                        797 non-null    float64
 6   dateTime                     797 non-null    object 
 7   receiptCode                  797 non-null    object 
 8   taxationType                 797 non-null    int64  
 9   requestNumber                797 non-null    int64  
 10  internetSign                 797 non-null    object 
 11  senderAddress                797 non-null    object 
 12  fiscalDocumentNumber         797 non-null    int64  
 13  buyerAddress        

## fiscalDocumentNumber

Рассмотрим столбец fiscalDocumentNumber. Судя по названию столбца, он должен содержать уникальные значения. Но это не так, 2 значения повторяются.

In [ ]:
data['fiscalDocumentNumber'].value_counts()

107334    2
141032    2
23206     1
6819      1
22510     1
         ..
97432     1
97447     1
97475     1
54481     1
29773     1
Name: fiscalDocumentNumber, Length: 795, dtype: int64

Посмотрим, в каких строках повторяются fiscalDocumentNumber.

In [ ]:
data.query('fiscalDocumentNumber == 107334')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,fiscalDocumentNumber,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value
674,Missing,1,20,11382,11382,0.0,2020-09-14,3.0,0,2461,Missing,Missing,107334,+79192540653,1897.0,0.0,Missing,Missing,0.0,Missing,Missing,Missing,Missing,0.0,0.0,0.0,Missing,0.0,Missing,Missing,Missing
738,Missing,1,20,15000,15000,0.0,2020-09-14,3.0,0,1897,Missing,Missing,107334,+79636175730,2500.0,0.0,Missing,Missing,0.0,Missing,Missing,Missing,Missing,0.0,0.0,0.0,Missing,0.0,Missing,Missing,Missing


In [ ]:
data.query('fiscalDocumentNumber == 141032')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,fiscalDocumentNumber,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value
521,Missing,3,198,7500,7500,0.0,2020-09-14,3.0,1,3179,Missing,Missing,141032,Missing,0.0,7500.0,Missing,Missing,0.0,noreply@chek.pofd.ru,Missing,Missing,Missing,0.0,0.0,0.0,"127410, Москва г, Алтуфьевское ш, дом № 33Г",0.0,Missing,Missing,Missing
671,Missing,1,616,7000,0,0.0,2020-09-14,3.0,1,100,Missing,Missing,141032,Missing,0.0,0.0,Missing,Missing,0.0,Missing,Missing,Missing,Missing,636.0,0.0,0.0,Missing,0.0,Missing,Missing,Missing


In [ ]:
data.drop('fiscalDocumentNumber', axis=1, inplace=True)

Вопреки моим предположениям, эти строки не являются копиями. Вероятно, у этой аномалии есть какое-то объяснение в рамках предметной области. Оставлю этот столбец как есть.

## Правка типов данных

Несколько числовых столбцов содержат данные, которые по смыслу являются категориальными. Изменим тип этих столбцов на object.

In [ ]:
columns_to_convert_to_categorical = ['operationType', 'taxationType']#, 'fiscalDocumentNumber']

In [ ]:
data[columns_to_convert_to_categorical] = data[columns_to_convert_to_categorical].astype('object')

In [ ]:
data['dateTime'] = pd.to_datetime(data['dateTime'])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   protocolVersion              797 non-null    object        
 1   operationType                797 non-null    object        
 2   shiftNumber                  797 non-null    int64         
 3   totalSum                     797 non-null    int64         
 4   ecashTotalSum                797 non-null    int64         
 5   nds18                        797 non-null    float64       
 6   dateTime                     797 non-null    datetime64[ns]
 7   receiptCode                  797 non-null    object        
 8   taxationType                 797 non-null    object        
 9   requestNumber                797 non-null    int64         
 10  internetSign                 797 non-null    object        
 11  senderAddress                797 non-null    

## Столбцы НДС

Посмотрим, как столбцы коррелируют друг с другом.

В первую очередь бросается в глаза очень сильная корреляция столбцов totalSum и ndsNo.

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,shiftNumber,totalSum,ecashTotalSum,nds18,requestNumber,ndsCalculated20,ndsNo,ndsCalculated18,nds10,nds0,nds20,ndsCalculated10
shiftNumber,1.000000,-0.028785,-0.031699,0.016790,-0.330815,-0.153055,-0.030696,-0.038265,0.154555,0.032261,0.034830,-0.019608
totalSum,-0.028785,1.000000,0.203236,0.044135,-0.040753,-0.006819,0.995667,-0.000771,0.003841,0.043443,-0.005680,-0.003213
ecashTotalSum,-0.031699,0.203236,1.000000,0.280909,0.024014,0.033263,0.168654,0.050040,0.033900,0.220465,-0.003741,-0.009308
nds18,0.016790,0.044135,0.280909,1.000000,0.096009,-0.072572,-0.029225,-0.064414,0.013338,-0.026196,-0.025029,-0.016006
requestNumber,-0.330815,-0.040753,0.024014,0.096009,1.000000,0.225620,-0.048112,-0.024896,-0.084272,-0.039630,-0.029848,-0.023602
ndsCalculated20,-0.153055,-0.006819,0.033263,-0.072572,0.225620,1.000000,-0.015440,-0.034020,-0.028452,-0.013836,-0.013219,-0.008454
ndsNo,-0.030696,0.995667,0.168654,-0.029225,-0.048112,-0.015440,1.000000,-0.011914,-0.008600,-0.005573,-0.005325,-0.003405
ndsCalculated18,-0.038265,-0.000771,0.050040,-0.064414,-0.024896,-0.034020,-0.011914,1.000000,-0.025254,-0.012281,-0.011733,-0.007503
nds10,0.154555,0.003841,0.033900,0.013338,-0.084272,-0.028452,-0.008600,-0.025254,1.000000,-0.010270,0.039146,-0.006275
nds0,0.032261,0.043443,0.220465,-0.026196,-0.039630,-0.013836,-0.005573,-0.012281,-0.010270,1.000000,-0.004772,-0.003052


Посмотрим, какие именно данные содержатся в столбцах, связанных с НДС.

In [ ]:
data.eval('ndsNo / totalSum').value_counts()

0.000000    515
1.000000    258
0.772302      1
0.246231      1
0.898058      1
0.898190      1
0.914961      1
1.000167      1
1.000040      1
dtype: int64

In [ ]:
data.eval('nds0 / totalSum').value_counts()

0.0    704
1.0     76
dtype: int64

Среди них есть столбцы с очень похожими названиями. Более того, эти столбцы никогда не заполнены одновременно.

In [ ]:
data.query('nds0 != 0 and ndsNo != 0')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value


In [ ]:
data.query('nds10 != 0 and ndsCalculated10 != 0')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value


In [ ]:
data.query('nds18 != 0 and ndsCalculated18 != 0')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value


In [ ]:
data.query('nds20 != 0 and ndsCalculated20 != 0')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,buyerAddress,ndsCalculated20,ndsNo,userProperty.key,userProperty.value,ndsCalculated18,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds0,nds20,retailAddress,ndsCalculated10,code,propertiesUser_key,propertiesUser_value


Объединим эти столбцы.

In [ ]:
data['ndsNo'] = data.apply(lambda x: x['ndsNo'] if x['ndsNo'] != 0 else x['nds0'], axis=1)
data.drop('nds0', axis=1, inplace=True)

data['nds10'] = data.apply(lambda x: x['nds10'] if x['nds10'] != 0 else x['ndsCalculated10'], axis=1)
data.drop('ndsCalculated10', axis=1, inplace=True)

data['nds18'] = data.apply(lambda x: x['nds18'] if x['nds18'] != 0 else x['ndsCalculated18'], axis=1)
data.drop('ndsCalculated18', axis=1, inplace=True)

data['nds20'] = data.apply(lambda x: x['nds20'] if x['nds20'] != 0 else x['ndsCalculated20'], axis=1)
data.drop('ndsCalculated20', axis=1, inplace=True)

Похоже, столбец ndsNo (и объединенный с ним столбец nds0), по идее, должны хранить сумму без НДС.

In [ ]:
data.query('nds10 != 0').shape[0]

48

In [ ]:
data.query('nds18 != 0').shape[0]

331

In [ ]:
data.query('nds20 != 0').shape[0]

88

Однако в строках, где НДС ненулевой, столбец ndsNo практически не содержит значений.

Удалим этот столбец.

In [ ]:
data.query('ndsNo != 0 and (nds10 != 0 or nds18 != 0 or nds20 != 0)').shape[0]

5

In [ ]:
data.drop('ndsNo', axis=1, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   protocolVersion              797 non-null    object        
 1   operationType                797 non-null    object        
 2   shiftNumber                  797 non-null    int64         
 3   totalSum                     797 non-null    int64         
 4   ecashTotalSum                797 non-null    int64         
 5   nds18                        797 non-null    float64       
 6   dateTime                     797 non-null    datetime64[ns]
 7   receiptCode                  797 non-null    object        
 8   taxationType                 797 non-null    object        
 9   requestNumber                797 non-null    int64         
 10  internetSign                 797 non-null    object        
 11  senderAddress                797 non-null    

В результате столбцы имеют такую корреляцию.

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,shiftNumber,totalSum,ecashTotalSum,nds18,requestNumber,nds10,nds20
shiftNumber,1.000000,-0.028785,-0.031699,0.007937,-0.330815,0.150039,-0.148594
totalSum,-0.028785,1.000000,0.203236,0.043463,-0.040753,0.003322,-0.007405
ecashTotalSum,-0.031699,0.203236,1.000000,0.289072,0.024014,0.032161,0.032709
nds18,0.007937,0.043463,0.289072,1.000000,0.089289,0.004776,-0.082073
requestNumber,-0.330815,-0.040753,0.024014,0.089289,1.000000,-0.086931,0.221376
nds10,0.150039,0.003322,0.032161,0.004776,-0.086931,1.000000,-0.025136
nds20,-0.148594,-0.007405,0.032709,-0.082073,0.221376,-0.025136,1.000000


In [ ]:
#list1 = list(data.select_dtypes(['object']).columns)
#list1

In [ ]:
#categorical_attributes_list = list1[2:len(list1)]#list(data.select_dtypes(['object']).columns)
#categorical_attributes_list.remove('taxationType')
#categorical_attributes_list.append('dateTime')
#.drop(categorical_attributes_list, axis=1)

In [ ]:
#data1 = data.copy()

## Подготовка к генерации похожих данных

In [ ]:
data['dateTime'].value_counts()

2020-09-14    525
2020-09-16    272
Name: dateTime, dtype: int64

Изменим формат представления даты. Это сделано для обхода бага в библиотеке генерации похожих данных.

Баг-репорт, написанный мной:

https://github.com/DataResponsibly/DataSynthesizer/issues/37

In [ ]:
#data1 = data.copy()
#data1['receiptCode'] = data1['receiptCode'].replace({3:'3,0', 'Missing':'Missing'})
#data1['dateTime'] = data1['dateTime'].replace({'2020-09-14':'20200914', '2020-09-16':'20200916'})
#print(data1['receiptCode'].value_counts())
#print(data1['dateTime'].value_counts())
#data1.drop('dateTime', axis=1, inplace=True)

20200914    525
20200916    272
Name: dateTime, dtype: int64


In [ ]:
#data1 = data.copy()
#index_list = list(range(data1.shape[0]))
#index_list[0] = 285
#index_list[285] = 0
#data1 = data1.reindex(index_list)
#data1.drop('dateTime', axis=1, inplace=True)
#data1.head()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   protocolVersion              797 non-null    object        
 1   operationType                797 non-null    object        
 2   shiftNumber                  797 non-null    int64         
 3   totalSum                     797 non-null    int64         
 4   ecashTotalSum                797 non-null    int64         
 5   nds18                        797 non-null    float64       
 6   dateTime                     797 non-null    datetime64[ns]
 7   receiptCode                  797 non-null    object        
 8   taxationType                 797 non-null    object        
 9   requestNumber                797 non-null    int64         
 10  internetSign                 797 non-null    object        
 11  senderAddress                797 non-null    

In [ ]:
data.rename(columns={'userProperty.key': 'userProperty_key',  'userProperty.value': 'userProperty_value', 'propertiesUser.propertyName': 'propertiesUser_propertyName'}, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   protocolVersion              797 non-null    object        
 1   operationType                797 non-null    object        
 2   shiftNumber                  797 non-null    int64         
 3   totalSum                     797 non-null    int64         
 4   ecashTotalSum                797 non-null    int64         
 5   nds18                        797 non-null    float64       
 6   dateTime                     797 non-null    datetime64[ns]
 7   receiptCode                  797 non-null    object        
 8   taxationType                 797 non-null    object        
 9   requestNumber                797 non-null    int64         
 10  internetSign                 797 non-null    object        
 11  senderAddress                797 non-null    

## Сохранение предобработанных данных

Сохраним предобработанные данные в файл.

In [ ]:
data.to_csv('/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/fns_fiscal_preprocessed.csv', index=False)

In [ ]:
#data.query('receiptCode == "Missing"')

In [ ]:
#data1 = data.copy()
#data1.reset_index(inplace=True)
#data1.at[0, 'index'] = 285
#data1.at[285, 'index'] = 0
#data1.head(286).tail()
#data1.set_index('index', inplace=True)
#data1.head()

In [ ]:
#data1 = data.copy()
#row_0 = data1.iloc[0].copy()
#data1.iloc[0] = data1.iloc[285]
#data1.iloc[285] = row_0
#data1.head()

In [ ]:
#data['propertiesUser'].value_counts()

In [ ]:
#data['ndsNo'].value_counts(), data['nds0'].value_counts(),

In [ ]:
#data.query('not (ndsCalculated20.isnull() != nds20.isnull())', engine='python')

In [ ]:
#nds_count = not_na_count.loc[['nds18', 'ndsCalculated20', 'ndsNo', 'ndsCalculated18', 'nds10', 'nds0', 'nds20', 'ndsCalculated10']]
#nds_count, nds_count.sum()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 799
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   protocolVersion              797 non-null    object        
 1   operationType                797 non-null    object        
 2   shiftNumber                  797 non-null    int64         
 3   totalSum                     797 non-null    int64         
 4   ecashTotalSum                797 non-null    int64         
 5   nds18                        797 non-null    float64       
 6   dateTime                     797 non-null    datetime64[ns]
 7   receiptCode                  797 non-null    object        
 8   taxationType                 797 non-null    object        
 9   requestNumber                797 non-null    int64         
 10  internetSign                 797 non-null    object        
 11  senderAddress                797 non-null    

In [ ]:
data['requestNumber'].min(), data['requestNumber'].max()

(1, 5695)

In [ ]:
data.query('nds18 % 1 > 0')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds18,dateTime,receiptCode,taxationType,requestNumber,internetSign,senderAddress,buyerAddress,userProperty.key,userProperty.value,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser.propertyName,nds10,nds20,retailAddress,code,propertiesUser_key,propertiesUser_value


In [ ]:
data.requestNumber.value_counts()

1       13
6       10
12       9
13       8
4        8
        ..
838      1
935      1
930      1
1493     1
289      1
Name: requestNumber, Length: 493, dtype: int64